# Homework 2: Learning to Rank <a class="anchor" id="toptop"></a>

**Submission instructions**:
- The cells with the `# YOUR CODE HERE` denote that these sections are graded and you need to add your implementation.
- Please use Python 3.6.5 and `pip install -r requirements.txt` to avoid version issues.
- The notebook you submit has to have the student ids, separated by underscores (E.g., `12341234_12341234_12341234_hw1.ipynb`).
- This will be parsed by a regexp, **so please double check your filename**.
- Only one member of each group has to submit the file (**please do not compress the .ipynb file when you will submit it**) to canvas.
- **Make sure to check that your notebook runs before submission**. A quick way to do this is to restart the kernel and run all the cells.  
- Do not change the number of arugments in the given functions.
- **Please do not delete/add new cells**. Removing cells **will** lead to grade deduction. 
- Note, that you are not allowed to use Google Colab.

**Learning Goals**:
- Offline LTR
  - Learn how to implement pointwise, pairwise and listwise algorithms for learning to rank 

---
**Recommended Reading**:
  - Chris Burges, Tal Shaked, Erin Renshaw, Ari Lazier, Matt Deeds, Nicole Hamilton, and Greg Hullender. Learning to rank using gradient descent. InProceedings of the 22nd international conference on Machine learning, pages 89–96, 2005.
  - Christopher J Burges, Robert Ragno, and Quoc V Le. Learning to rank with nonsmooth cost functions. In Advances inneural information processing systems, pages 193–200, 2007
  - (Sections 1, 2 and 4) Christopher JC Burges. From ranknet to lambdarank to lambdamart: An overview. Learning, 11(23-581):81, 2010
  

Additional Resources: 
- This assignment requires knowledge of [PyTorch](https://pytorch.org/). If you are unfamiliar with PyTorch, you can go over [these series of tutorials](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)

In the previous assignment, you experimented with retrieval with different ranking functions and in addition, different document representations. 

This assignment deals directly with learning to rank (LTR). In offline LTR, You will learn how to implement methods from the three approaches associated with learning to rank: pointwise, pairwise and listwise. 


**Note:**
  - The dataset used in this assignment is +100Mb in size. You may need around 2Gb of RAM for running the whole notebook.


# Table of Contents  <a class="anchor" id="top"></a>

[Back to top](#toptop)


Table of contents:


 - [Chapter 1: Offline LTR](#o_LTR) (270 points)
     - [Section 1: Dataset and Utility](#dataU) (- points)
     - [Section 2: Pointwtise LTR](#pointwiseLTR) (55 points)
     - [Section 3: Pairwise LTR](#pairwiseLTR) (35 points)
     - [Section 4: Pairwise Speed-up RankNet](#SpairwiseLTR) (65 points)
     - [Section 5: Listwise LTR](#listwiseLTR) (60 points)
     - [Section 6: Evaluation](#evaluation1) (55 points)

In [ ]:
import os
import json
import itertools
from argparse import Namespace
from collections import OrderedDict
from functools import partial


import torch
import numpy as np
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange
from torch.utils.data import Dataset, DataLoader


import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

import pandas as pd

import dataset
import evaluate


# Chapter 1: Offline LTR <a class="anchor" id="o_LTR"></a>

A typical setup of learning to rank involves a feature vector constructed using a query-document pair, and a set of relevance judgements. You are given a set of triples (`query`, `document`, `relevance grade`); where relevance grade is an *ordinal* variable  with  5  grades,  for example: {`perfect`,`excellent`,`good`,`fair`,`bad`),  typically  labeled  by human annotators.  

In this assignment, you are already given the feature vector for a given document and query pair. To access these vectors, see the following code cells (note: the dataset will be automatically downloaded & the first time the next cell runs, it will take a while!)

## Section 1: Data and Utility <a class="anchor" id="dataU"></a>

[Back to TOC](#top)

First let's get familiar with the dataset and some utility methods for our implementations.

### Section 1.1 Dataset stats

| Split Name | \# queries | \# docs | \# features |
| :- | :--: | :--: | :--: |
| train | 2735 | 85227 | 501 |
| validation | 403 | 12794 | 501 |
| test | 949 | 29881 | 501 |


In [ ]:
dataset.download_dataset()
data = dataset.get_dataset()
# there is only 1 fold for this dataset 
data = data.get_data_folds()[0]
# read in the data
data.read_data()

In [ ]:
print(f"Number of features: {data.num_features}")
# print some statistics
for split in ["train", "validation", "test"]:
    print(f"Split: {split}")
    split = getattr(data, split)
    print(f"\tNumber of queries {split.num_queries()}")
    print(f"\tNumber of docs {split.num_docs()}")

### Section 1.2 Utility classes/methods

The following cells contain code that will be useful for the assigment. 

In [ ]:
# these is a useful class to create torch DataLoaders, and can be used during training
class LTRData(Dataset):
    def __init__(self, data, split):
        split = {
            "train": data.train,
            "validation": data.validation,
            "test": data.test
        }.get(split)
        assert split is not None, "Invalid split!"
        features, labels = split.feature_matrix, split.label_vector
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return self.features.size(0)

    def __getitem__(self, i):
        return self.features[i], self.labels[i]

In [ ]:
## example 
train_dl = DataLoader(LTRData(data, "train"), batch_size=32, shuffle=True)
# this is how you would use it to quickly iterate over the train/val/test sets 
# - (of course, without the break statement!)
for (x, y) in train_dl:
    print(x.size(), y.size())
    break

`evaluate_model` evaluates a model, on a given split.

In [ ]:
# this function evaluates a model, on a given split
def evaluate_model(pred_fn, split, batch_size=256, print_results=False, q_level=False):
    dl = DataLoader(LTRData(data, split), batch_size=batch_size)
    all_scores = []
    all_labels = []
    for (x, y) in tqdm(dl, desc=f'Eval ({split})', leave=False):
        all_labels.append(y.squeeze().numpy())
        
        with torch.no_grad():
            output = pred_fn(x)
            all_scores.append(output.squeeze().numpy())
            
    split = {
            "train": data.train,
            "validation": data.validation,
            "test": data.test
    }.get(split)   
    results = evaluate.evaluate2(np.asarray(all_scores), np.asarray(all_labels), print_results=print_results, q_level=q_level)

    return results


In [ ]:
## example 
# function that scores a given feature vector e.g a network
net = nn.Linear(501, 1)
# the evaluate method accepts a function. more specifically, a callable (such as pytorch modules) 
def notwork(x):
    return net(x)
# evaluate the function
_ = evaluate_model(notwork, "validation", print_results=True)

The next cell is used to generate reproducible results:

In [ ]:
# use to get reproducible results
def seed(random_seed):
    import random
    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    

## Section 2: Pointwise LTR (55 points) <a class="anchor" id="pointwiseLTR"></a>

[Back to TOC](#top)

Let $x \in \mathbb{R}^d$ be an input feature vector, containing features for a query-document pair. Let $f: \mathbb{R}^d \rightarrow \mathbb{R} $ be a function that maps this feature vector to a number $f(x)$ - either a relevance score (regression) or label (classification). The data $\{x \}$ are treated as feature vectors and the relevance judgements are treated as the target which we want to predict. 

In this section, you will implement a simple Pointwise model using either a regression loss, and use the train set to train this model to predict the relevance score. 


### Section 2.1: Neural Model (25 points)

In the following cell, you will implement a simple pointwise LTR model: 
- Use a neural network to learn a model with different loss functions, using the relevance grades as the label. Use the following parameters: 
  - Layers: $501 (input) \rightarrow 256 \rightarrow 1$ where each layer is a linear layer (`nn.Linear`) with a ReLu activation function (`nn.ReLU`) in between the layers. Use the default weight initialization scheme. (Hint: use `nn.Sequential` for a one-line forward function!)
  - This network will also be used by other methods i.e Pairwise 
  
You should implement the following three methods:
- `__init__` (4 points)
- `forward` (1 point)



In [ ]:
# TODO: Implement this! (5 points)
class NeuralModule(nn.Module):
    def __init__(self):
        """
        Initializes the Pointwise neural network.
        """
        
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def forward(self, x):
        """
        Takes in an input feature matrix of size (N, 501) and produces the output 
        Input: x: a [N, 501] tensor
        Output: a [N, 1] tensor
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        

In [ ]:
# check the network configuration - layer dimensions and configurations
point_nn_reg = NeuralModule()
print(point_nn_reg)

In [ ]:
# test the forward function
n = 10
inp = torch.rand(n, data.num_features)
out = point_nn_reg(inp)


**Implementation (20 points):**
Implement `train_batch` function to compute the gradients (`backward()` function) and update the weights (`step()` function), using the specified loss function.

In [ ]:
# TODO: Implement this! (20 points)

def train_batch(net, x, y, loss_fn, optimizer):
    """
    Takes as input a batch of size N, i.e. feature matrix of size (N, 501), label vector of size (N), the loss function and optimizer for computing the gradients, and updates the weights of the model.

    Input:  x: feature matrix, a [N, 501] tensor
            y: label vector, a [N] tensor
            loss_fn: an implementation of a loss function
            optimizer: an optimizer for computing the gradients (we use Adam)
    """
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
# Please do not change this. This cell is used for grading.


\#### Please do not change this. This cell is used for grading.

### Section 2.2: Loss Functions (5 points)
Pointwise LTR algorithms use pointwise loss functions.
Usually, the popular loss functions for pointwise LTR is Regression loss.

**Implementation (5 points):**
Implement regression loss.

In [ ]:
# TODO: Implement this! (5 points)
def pointwise_loss(output, target):
    """
    Regression loss - returns a single number. 
    Make sure to use the MSE loss
    output: (float) tensor, shape - [N, 1] 
    target: (float) tensor, shape - [N]. 
    """
    assert target.dim() == 1
    assert output.size(0) == target.size(0)
    assert output.size(1) == 1
    
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
## Test pointwise_loss
g = torch.manual_seed(42)
output = [torch.randint(low=0, high=5, size=(5, 1), generator=g).float() for _ in range(5)]
target = torch.randint(low=0, high=5, size=(5,), generator=g).float()

l = [pointwise_loss(o, target).item() for o in output]
print(f'your results:{l}')


**Implementation (25 points):**
Now implement a wrapper for training a pointwise LTR, that takes the model as input and trains the model.

**Rubric:**
 - Network is trained for specified epochs, and iterates over the entire dataset and (train) data is shuffled : 5 points
 - Evaluation on the validation set: 5 points
 - Performance as expected: 15 points

In [ ]:
# TODO: Implement this! (25 points)
def train_pointwise(net, params):
    """
    This function should train a Pointwise network. 
    
    The network is trained using the Adam optimizer
        
    
    Note: Do not change the function definition! 
    
    
    Hints:
    1. Use the LTRData class defined above
    2. Do not forget to use net.train() and net.eval()
    
    Inputs:
            net: the neural network to be trained

            params: params is an object which contains config used in training 
                (eg. params.epochs - the number of epochs to train). 
                For a full list of these params, see the next cell. 
    
    Returns: a dictionary containing: "metrics_val" (a list of dictionaries) and 
             "metrics_train" (a list of dictionaries). 
             
             "metrics_val" should contain metrics (the metrics in params.metrics) computed
             after each epoch on the validation set (metrics_train is similar). 
             You can use this to debug your models.
    
    """
    
    
    val_metrics_epoch = []
    train_metrics_epoch = []
    optimizer = Adam(net.parameters(), lr=params.lr)
    loss_fn = pointwise_loss
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return {
        "metrics_val": val_metrics_epoch,
        "metrics_train": train_metrics_epoch
    }

In [ ]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [ ]:
# Change this to test your code!
pointwise_test_params = Namespace(epochs=1, 
                    lr=1e-3,
                    batch_size=256,
                   metrics={"ndcg"})
# uncomment to test your code
## train a regression model
# met_reg = train_pointwise(point_nn_reg, pointwise_test_params)


The next cell is used to generate results:

In [ ]:

    
def create_results(net, train_fn, prediction_fn, *train_params):
    
    print("Training Model")
    metrics = train_fn(net, *train_params)
    net.eval()
    test_metrics, test_qq = evaluate_model(prediction_fn, "test", print_results=True, q_level=True)
    
    
    test_q = {}
    for m in {"ndcg", "precision@05", "recall@05"}:
        test_q[m] = test_qq[m]
    
    
    return {
        "metrics": metrics,
        "test_metrics": test_metrics,
        "test_query_level_metrics": test_q,
        }


Now use the above functions to generate your results:

In [ ]:
seed(42)
params_regr = Namespace(epochs=11, 
                    lr=1e-3,
                    batch_size=256,
                    metrics={"ndcg", "precision@05", "recall@05"})

pointwise_regression_model = NeuralModule()
pw_regr = create_results(pointwise_regression_model, 
                          train_pointwise, 
                          pointwise_regression_model,
                          params_regr)

In [ ]:
assert "test_metrics" in pw_regr.keys()
assert "ndcg" in pw_regr["test_metrics"].keys()
assert "precision@05" in pw_regr["test_metrics"].keys()
assert "recall@05" in pw_regr["test_metrics"].keys()

## Section 3: Pairwise LTR (35 points) <a class="anchor" id="pairwiseLTR"></a>

[Back to TOC](#top)

In this section,  you will learn and implement RankNet, a  pairwise learning to rank algorithm.

For a given query, consider two documents $D_i$ and $D_j$ with two different ground truth relevance  labels,  with  feature  vectors $x_i$ and $x_j$ respectively.   The  RankNet  model,  just  like  the pointwise model, uses $f$ to predict scores i.e $s_i=f(x_i)$ and $s_j=f(x_j)$, but uses a different loss during  training. $D_i \triangleright D_j$ denotes  the  event  that $D_i$ should  be  ranked  higher  than $D_j$.   The  two outputs $s_i$ and $s_j$ are mapped to a learned probability that $D_i \triangleright D_j$: 


$$        P_{ij} = \frac{1}{1 + e^{-\sigma(s_i - s_j)}} $$
  
where $\sigma$ is a parameter that determines the shape of the sigmoid. The loss of the RankNet model is the cross entropy cost function:

$$        C = - \bar{P}_{ij} \log P_{ij} - (1-\bar{P}_{ij}) \log (1 - P_{ij}) $$

As the name suggests, in the pairwise approach to LTR, we optimize a loss $l$ over pairs of documents. Let $S_{ij} \in \{0, \pm1 \}$ be equal to $1$ if the relevance of document $i$ is greater than document $j$; $-1$ if document $j$ is more relevant than document $i$; and 0 if they have the same relevance. This gives us $\bar{P}_{ij} = \frac{1}{2} (1 + S_{ij})$ so that $\bar{P}_{ij} = 1$ if $D_i \triangleright D_j$; $\bar{P}_{ij} = 0$ if $D_j \triangleright D_i$; and finally $\bar{P}_{ij} = \frac{1}{2}$ if the relevance is identical. This gives us:

$$        C = \frac{1}{2}(1- S_{ij})\sigma(s_i - s_j) + \log(1+ e^{-\sigma(s_i - s_j)}) $$

Now, consider a single query for which $n$ documents have been returned. Let the output scores of the ranker be $s_j$ ; $j=\{1, \dots, n \}$, the model parameters be $w_k \in \mathbb{R}^W$, and let the set of pairs of document indices used for training be $\mathcal{P}$. Then, the total cost is $C_T = \sum_{i,j \in \mathcal{P}} C(s_i; s_j)$. 



- Implement RankNet. You should construct training samples by creating all possible pairs of documents for a given query and optimizing the loss above. Use the following parameters:
  - Layers: $501 (input) \rightarrow 256 \rightarrow 1$, where each layer is a linear layer (`nn.Linear`) with a ReLu activation function (`nn.ReLu`) in between the layers. Use the default weight initialization scheme. (Hint: use `nn.Sequential` for a one-line forward function!)

For the pairwise loss, we need to have a structured **dataloader** which detects the documents associated with a specific query:

In [ ]:
class QueryGroupedLTRData(Dataset):
    def __init__(self, data, split):
        self.split = {
            "train": data.train,
            "validation": data.validation,
            "test": data.test
        }.get(split)
        assert self.split is not None, "Invalid split!"
    
    def __len__(self):
        return self.split.num_queries()

    def __getitem__(self, q_i):
        feature = torch.FloatTensor(self.split.query_feat(q_i))
        labels = torch.FloatTensor(self.split.query_labels(q_i))
        return feature, labels

    
## example
train_data = QueryGroupedLTRData(data, "train")
# this is how you would use it to quickly iterate over the train/val/test sets 

q_i = 300
features_i, labels_i = train_data[q_i]
print(f"Query {q_i} has {len(features_i)} query-document pairs")
print(f"Shape of features for Query {q_i}: {features_i.size()}")
print(f"Shape of labels for Query {q_i}: {labels_i.size()}")
        

**Implementation (35 points):**
First, implement the pairwaise loss, described above.

**Rubric:**
 - Each ordering <i,j> combination is considered: 10 points
 - Proper application of the formula: 10 points
 - Mean loss: 5 points
 - Loss values for test cases as expected: 10 points

In [ ]:
# TODO: Implement this! (35 points)
def pairwise_loss(scores, labels):
    """
    Compute and return the pairwise loss *for a single query*. To compute this, compute the loss for each 
    ordering in a query, and then return the mean. Use sigma=1.
    
    For a query, consider all possible ways of comparing 2 document-query pairs.
    
    Hint: See the next cell for an example which should make it clear how the inputs look like
    
    scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
    labels: tensor of size [N], contains the relevance labels 
    
    """
    # if there's only one rating
    if labels.size(0) < 2:
        return None
    # YOUR CODE HERE
    raise NotImplementedError()
    

In [ ]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [ ]:
scores_1 = torch.FloatTensor([0.2, 2.3, 4.5, 0.2, 1.0]).unsqueeze(1)
labels_1 = torch.FloatTensor([1, 2, 3, 0, 4])


scores_2 = torch.FloatTensor([3.2, 1.7]).unsqueeze(1)
labels_2 = torch.FloatTensor([3, 1])

assert torch.allclose(pairwise_loss(scores_1, labels_1), torch.tensor(0.6869), atol=1e-03)
assert torch.allclose(pairwise_loss(scores_2, labels_2), torch.tensor(0.2014), atol=1e-03)


## Section 4: Pairwise: Speed-up RankNet (65 points) <a class="anchor" id="SpairwiseLTR"></a>

[Back to TOC](#top)

To speed up training of the previous model, we can consider a sped up version of the model, where instead of `.backward` on the loss, we use `torch.backward(lambda_i)`. 

The derivative of the total cost $C_T$ with respect to the model parameters $w_k$ is:

$$        \frac{\partial C_T}{\partial w_k} = \sum_{(i,j) \in \mathcal{P}} \frac{\partial C(s_i, s_j)}{\partial s_i} \frac{\partial s_i}{\partial w_k} + \frac{\partial C(s_i, s_j)}{\partial s_j} \frac{\partial s_j}{\partial w_k} $$

We can rewrite this sum by considering the set of indices $j$ , for which $\{i,j\}$ is a valid pair, denoted by $\mathcal{P}_i$, and the set of document indices $\mathcal{D}$:

$$
\frac{\partial C_T}{\partial w_k} = \sum_{i \in \mathcal{D}}
\frac{\partial s_i}{\partial w_k} \sum_{j \in \mathcal{P}_i} 
\frac{\partial C(s_i, s_j)}{\partial s_i} 
$$

This sped of version of the algorithm first computes scores $s_i$ for all the documents. Then for each $j= 1, \dots, n$, compute:

$$
\lambda_{ij} = \frac{\partial C(s_i, s_j)}{\partial s_i} = \sigma \bigg( \frac{1}{2}(1 - S_{ij}) -  \frac{1}{1 + e^{\sigma(s_i -s_j))}} \bigg) \\
\lambda_i = \sum_{j \in \mathcal{P}_i} \frac{\partial C(s_i, s_j)}{\partial s_i} = \sum_{j \in \mathcal{P}_i} \lambda_{ij}
$$

That gives us:

$$
\frac{\partial C_T}{\partial w_k} = \sum_{i \in \mathcal{D}}
\frac{\partial s_i}{\partial w_k} \lambda_i
$$

This can be directly optimized in pytorch using: `torch.autograd.backward(scores, lambda_i)` 
 


**Implementation (50 points):**
Implement the sped-up version of pairwise loss, described above.

**Rubric:**
 - Each ordering <i,j> combination is considered: 15 points
 - Proper application of the formula: 15 points
 - Loss values for test cases as expected: 20 points

In [ ]:
# TODO: Implement this! (50 points)
def compute_lambda_i(scores, labels):
    """
    Compute \lambda_i (defined in the previous cell). (assume sigma=1.)
    
    scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
    labels: tensor of size [N], contains the relevance labels 
    
    return: \lambda_i, a tensor of shape: [N, 1]
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [ ]:
def mean_lambda(scores, labels):
    return torch.stack([compute_lambda_i(scores, labels).mean(), torch.square(compute_lambda_i(scores, labels)).mean()])

scores_1 = torch.FloatTensor([10.2, 0.3, 4.5, 2.0, -1.0]).unsqueeze(1)
labels_1 = torch.FloatTensor([1, 2, 3, 0, 4])

assert torch.allclose(mean_lambda(scores_1, labels_1), torch.tensor([0,5.5072]), atol=1e-03)

scores_2 = torch.FloatTensor([3.2, 1.7]).unsqueeze(1)
labels_2 = torch.FloatTensor([3, 1])

assert torch.allclose(mean_lambda(scores_2, labels_2), torch.tensor([0,3.3279e-02]), atol=1e-03)



**Implementation (15 points):**
Implement `train_batch_vector` function to compute the gradients (`torch.autograd.backward(scores, lambda_i)` function) and update the weights (`step()` function), using the specified loss function.

In [ ]:
# TODO: Implement this! (15 points)

def train_batch_vector(net, x, y, loss_fn, optimizer):
    """
    Takes as input a batch of size N, i.e. feature matrix of size (N, 501), label vector of size (N), the loss function and optimizer for computing the gradients, and updates the weights of the model.
    The loss function returns a vector of size [N, 1], the same as the output of network.

    Input:  x: feature matrix, a [N, 501] tensor
            y: label vector, a [N] tensor
            loss_fn: an implementation of a loss function
            optimizer: an optimizer for computing the gradients (we use Adam)
    """
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

##  Section 5: Listwise LTR (60 points) <a class="anchor" id="listwiseLTR"></a>

[Back to TOC](#top)

In this section, you will implement LambdaRank, a listwise approach to LTR. Consider the computation of $\lambda$ for sped-up RankNet (that you've already implemented). $\lambda$ here amounts to the 'force' on a document given its neighbours in the ranked list. The design of $\lambda$ in LambdaRank is similar to RankNet, but is scaled by DCG gain from swapping the two documents in question. Let's suppose that the corresponding ranks of doucment $D_i$ and $D_j$ are $r_i$ and $r_j$ respectively. Given a ranking measure $IRM$, such as $NDCG$ or $ERR$, the lambda function in LambdaRank is defined as:


$$        \frac{\partial C}{\partial s_i} = \sum_{j \in D} \lambda_{ij} \cdot |\bigtriangleup IRM (i,j)| $$

Where $|\bigtriangleup IRM(i,j)|$ is the absolute difference in $IRM$ after swapping the rank positions $r_i$ and $r_j$ while leaving everything else unchanged ($| \cdot |$ denotes the absolute value). Note that we do not backpropogate $|\bigtriangleup IRM|$, it is treated as a constant that scales the gradients. In this assignment we will use $|\bigtriangleup NDCG|$

**Implementation (60 points):**
Implement the listwise loss.

**Rubric:**
 - Each ordering <i,j> combination is considered: 15 points
 - Computing $|\bigtriangleup NDCG|$: 15 points 
 - Proper application of the formula: 15 points 
 - Loss values as expected: 15 points

In [ ]:
# TODO: Implement this! (60 points)
def listwise_loss(scores, labels):
    
    """
    Compute the LambdaRank loss. (assume sigma=1.)
    
    scores: tensor of size [N, 1] (the output of a neural network), where N = length of <query, document> pairs
    labels: tensor of size [N], contains the relevance labels 
    
    returns: a tensor of size [N, 1]
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

\#### Please do not change this. This cell is used for grading.

In [ ]:
def mean_lambda_list(scores, labels):
    return torch.stack([listwise_loss(scores, labels).mean(), torch.square(listwise_loss(scores, labels)).mean()])


scores_1 = torch.FloatTensor([10.2, 0.3, 4.5, 2.0, -1.0]).unsqueeze(1)
labels_1 = torch.FloatTensor([1, 2, 3, 0, 4])
assert torch.allclose(mean_lambda_list(scores_1, labels_1), torch.tensor([0,0.1391]), atol=1e-03)

scores_2 = torch.FloatTensor([3.2, 1.7]).unsqueeze(1)
labels_2 = torch.FloatTensor([3, 1])
assert torch.allclose(mean_lambda_list(scores_2, labels_2), torch.tensor([0,2.8024e-03]), atol=1e-03)



## Section 6: Comparing Pointwise, Pairwise and Listwise (55 points) <a class="anchor" id="evaluation1"></a>

[Back to TOC](#top)

In the next few cells, we will compare the methods you've implemented. Helper functions are provided for you, which you can use to make some conclusions. You can modify the code as needed!

First, let's have a function that plots the average scores of relevant (levels 3 and 4) and non-relevant (levels 0, 1, and 2) scores in terms of training epochs for different loss functions:

In [ ]:
loss_functions = {'pointwise':[pointwise_loss, train_batch],
                 'pairwise':[compute_lambda_i, train_batch_vector],
                 'listwise':[listwise_loss, train_batch_vector]}

def plot_relevance_scores(batches, loss_function):
    seed(420)
    net = NeuralModule()
    optimizer = Adam(net.parameters(), lr=0.005)
    loss_fn = loss_functions[loss_function][0]
    train_fn = loss_functions[loss_function][1]
    
    
    train_batchs = batches[:len(batches)*3//4]
    test_batchs = batches[len(batches)*3//4:]
    
    rel, nrel = [], []
    
    for i in range(100):
        r, n = [], []
        for x, y in test_batchs:
            binary_rel = np.round(y/4,0)
            scores = net(x)[:,0]
            r.append(torch.sum(scores * binary_rel).detach().numpy() / torch.sum(binary_rel).detach().numpy())
            n.append(torch.sum(scores * (1. - binary_rel)).detach().numpy() / torch.sum((1. - binary_rel)).detach().numpy())
            
        for x, y in train_batchs:
            train_fn(net, x, y, loss_fn, optimizer)
        rel.append(np.mean(np.array(r)))
        nrel.append(np.mean(np.array(n)))
        
    
        
    plt.figure()
    plt.suptitle(loss_function)
    plt.plot(np.arange(10,len(rel)), rel[10:], label='relevant')
    plt.plot(np.arange(10,len(nrel)), nrel[10:], label='non-relevant')
    plt.legend()
    
        

For efficiency issues, we select a small number (83) of queries to test different loss functions.
We split these queries into train and test with a 3:1 ratio.

In [ ]:
batches = [train_data[i] for i in [181, 209, 233, 242, 259, 273, 327, 333, 377, 393, 410, 434, 452, 503, 529, 573, 581, 597, 625, 658, 683, 724, 756, 757, 801, 825, 826, 828, 874, 902, 1581, 1588, 1636, 1691, 1712, 1755, 1813, 1983, 2001, 2018, 2021, 2024, 2029, 2065, 2095, 2100, 2171, 2172, 2174, 2252, 2274, 2286, 2288, 2293, 2297, 2353, 2362, 2364, 2365, 2368, 2400, 2403, 2433, 2434, 2453, 2472, 2529, 2534, 2539, 2543, 2555, 2576, 2600, 2608, 2636, 2641, 2653, 2692, 2714, 2717, 2718, 2723, 2724]]

Next, we train a neural network with different loss functions on the selected queries.
During training, we save the average scores of relevant and non-relevant validation items for each training epoch and plot them as follows:

In [ ]:

plot_relevance_scores(batches, 'pointwise')

plot_relevance_scores(batches, 'pairwise')

plot_relevance_scores(batches, 'listwise')

**Implementation (15 points):**
Now implement a function similar to `plot_relevance_scores` that measures the NDCG@10 on the test split with different loss functions.
Train your model for 10 epochs.
For NDCG@10 use `evaluate.ndcg10(scores.detach().numpy(), y.detach().numpy())` for each query and average through all queries to obtain NDCG@10 for each loss function at each epoch.

In [ ]:
# TODO: Implement this! (15 points)


def plot_ndcg10(batches, loss_function):
    seed(420)
    net = NeuralModule()
    optimizer = Adam(net.parameters(), lr=0.005)
    loss_fn = loss_functions[loss_function][0]
    train_fn = loss_functions[loss_function][1]
        
    train_batchs = batches[:len(batches)*3//4]
    test_batchs = batches[len(batches)*3//4:]
    
    ndcg = []
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
#     plt.figure()
    plt.plot(np.arange(len(ndcg)), ndcg, label=loss_function)
    plt.legend()
    
        
    


In [ ]:
# Please do not change this. This cell is used for grading.

\#### Please do not change this. This cell is used for grading.

In [ ]:

plot_ndcg10(batches, 'pointwise')

plot_ndcg10(batches, 'pairwise')

plot_ndcg10(batches, 'listwise')

Write a conclusion in the next cell, considering (40 points):
- rates of convergence
- time complexity
- distinguishing relevant and non-relevant items
- performance for low data wrt NDCG@10
- performance across queries
- ... any other observations

YOUR ANSWER HERE